In [3]:
import os
path = r'C:\Users\Nisarg Shah\Downloads\abs1Text\Text'
os.chdir(path)
text = []
data = []
for filename in os.listdir('.'):
    with open(filename,encoding="utf8") as myfile:
        data.append(myfile.read())
    text.append(data)
    
NO_DOCUMENTS = len(text)
print(NO_DOCUMENTS)
print(text[0])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
 
NUM_TOPICS = 10
STOPWORDS = stopwords.words('english')
 
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text
 
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in data:
    tokenized_data.append(clean_text(text))

# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]


In [5]:
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [6]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
print("=" * 20)

LDA Model:
Topic #0: 0.013*"performance" + 0.011*"management" + 0.006*"practices" + 0.006*"journal" + 0.005*"research" + 0.005*"environmental" + 0.005*"study" + 0.004*"social" + 0.004*"organizational" + 0.004*"also"
Topic #1: 0.014*"performance" + 0.010*"management" + 0.006*"journal" + 0.006*"human" + 0.005*"practices" + 0.005*"research" + 0.005*"study" + 0.004*"resource" + 0.004*"organizational" + 0.004*"social"
Topic #2: 0.011*"performance" + 0.009*"management" + 0.006*"environmental" + 0.005*"corporate" + 0.004*"human" + 0.004*"social" + 0.004*"study" + 0.004*"journal" + 0.004*"practices" + 0.004*"research"
Topic #3: 0.013*"performance" + 0.009*"management" + 0.006*"journal" + 0.005*"employees" + 0.005*"turnover" + 0.004*"may" + 0.004*"practices" + 0.004*"study" + 0.004*"business" + 0.004*"human"
Topic #4: 0.014*"performance" + 0.009*"management" + 0.006*"practices" + 0.006*"organizational" + 0.005*"journal" + 0.005*"environmental" + 0.004*"human" + 0.004*"firm" + 0.004*"work" + 0.0

In [ ]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

datafiles = load_files(r"C:\Users\Nisarg Shah\Desktop\abs1Text\abs1Text - Copy\Text")

X,y = datafiles.data, datafiles.target

In [3]:
documents = []

from nltk.stem import WordNetLemmatizer
import re

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = vectorizer.fit_transform(documents).toarray()

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer  
tfidfconverter = TfidfTransformer()  
X = tfidfconverter.fit_transform(X).toarray()  

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
classifier.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [16]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0])

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

[[16  5]
 [ 4  8]]
             precision    recall  f1-score   support

          0       0.80      0.76      0.78        21
          1       0.62      0.67      0.64        12

avg / total       0.73      0.73      0.73        33

0.7272727272727273
